# Basic PySyft Setup

### RecNN prerequisites 

In [3]:
import numpy as np
import pandas as pd
import torch
import syft as sy
from tqdm.auto import tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


# == recnn ==
import sys
sys.path.append("../../")
import recnn

cuda = torch.device('cuda')

# ---
frame_size = 10
batch_size = 25
n_epochs   = 100
plot_every = 30
step       = 0
# --- 

tqdm.pandas()

from jupyterthemes import jtplot
jtplot.style(theme='grade3')

In [4]:
# embeddgings: https://drive.google.com/open?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
env = recnn.data.env.FrameEnv('../../data/embeddings/ml20_pca128.pkl',
                         '../../data/ml-20m/ratings.csv', frame_size, batch_size, num_workers=0)

In [4]:
def overwrite_batch_tensor_embeddings(batch, item_embeddings_tensor, frame_size):
    
    from recnn.data.utils import get_irsu
    items_t, ratings_t, sizes_t, users_t = get_irsu(batch)
    
    print(items_t.long()[0])
    print(item_embeddings_tensor[items_t.long()])
    items_emb = item_embeddings_tensor[items_t.long()]
    b_size = ratings_t.size(0)

    items = items_emb[:, :-1, :].view(b_size, -1)
    next_items = items_emb[:, 1:, :].view(b_size, -1)
    ratings = ratings_t[:, :-1]
    next_ratings = ratings_t[:, 1:]

    state = torch.cat([items, ratings], 1)
    next_state = torch.cat([next_items, next_ratings], 1)
    action = items_emb[:, -1, :]
    reward = ratings_t[:, -1]

    done = torch.zeros(b_size)
    # for some reason syft dies at this line
    # so no done in training. not a big deal
    # done[torch.cumsum(sizes_t - frame_size, dim=0) - 1] = 1

    batch = {'state': state, 'action': action, 'reward': reward, 'next_state': next_state, 'done': done,
             'meta': {'users': users_t, 'sizes': sizes_t}}
    return batch

recnn.data.utils.batch_tensor_embeddings = overwrite_batch_tensor_embeddings

In [5]:
# test function
def run_tests():
    batch = next(iter(env.test_dataloader))
    loss = ddpg.update(batch, learn=False)
    return loss

value_net  = recnn.nn.Critic(1290, 128, 256, 54e-2)
policy_net = recnn.nn.Actor(1290, 128, 256, 6e-1)
ddpg = recnn.nn.DDPG(policy_net, value_net)
plotter = recnn.utils.Plotter(ddpg.loss_layout, [['value', 'policy']],)

## Syft

In [8]:
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

# share embeddings
env.embeddings = env.embeddings.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [9]:
env.train_dataloader = sy.FederatedDataLoader(env.train_user_dataset.federate((james, bob, alice)),
                                              batch_size=env.batch_size,
                                              shuffle=True,
                                              num_workers=env.num_workers,
                                              collate_fn=env.prepare_batch_wrapper)

env.test_dataloader = sy.FederatedDataLoader(env.test_user_dataset.federate((james, bob, alice)),
                                              batch_size=env.batch_size,
                                              shuffle=True,
                                              num_workers=env.num_workers,
                                              collate_fn=env.prepare_batch_wrapper)

AttributeError: Could not find inputs in dataset

In [6]:
from collections import defaultdict
targets = defaultdict(lambda: torch.tensor(0))
env.test_user_dataset_fed = sy.BaseDataset(env.test_user_dataset, targets)
env.train_user_dataset_fed = sy.BaseDataset(env.train_user_dataset, targets)

In [9]:
env.test_user_dataset_fed.federate((james, bob, alice))

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 3958 and 3742 in dimension 1 at /pytorch/aten/src/TH/generic/THTensor.cpp:689

In [71]:
env.embeddings.size()

torch.Size([0])

In [72]:
share = lambda  m: m.fix_precision().share(bob, alice,crypto_provider=james, requires_grad=True)
ddpg.nets = dict([(k, share(v)) for k, v in ddpg.nets.items()])

In [73]:
share_optim = lambda o: o.fix_precision()
ddpg.optimizers =  dict([(k, share_optim(v)) for k, v in ddpg.optimizers.items()])

In [74]:
print(env.embeddings)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:71653941824 -> bob:28508861448]
	-> [PointerTensor | me:12220294150 -> alice:72311280919]
	*crypto provider: james*


In [75]:
print(ddpg.nets['policy_net'])

Actor(
  (drop_layer): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=1290, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=128, bias=True)
)


In [76]:
plot_every = 50
n_epochs = 2

def learn():
    for epoch in range(n_epochs):
        for batch in tqdm(env.train_dataloader):
            continue 
            loss = ddpg.update(batch, learn=True)
            plotter.log_losses(loss)
            ddpg.step()
            if ddpg._step % plot_every == 0:
                clear_output(True)
                print('step', ddpg._step)
                test_loss = run_tests()
                plotter.log_losses(test_loss, test=True)
                plotter.plot_loss()
            if ddpg._step > 1000:
                return
            
learn()

tensor([148, 586, 584, 293, 376, 345, 163, 582, 340, 151, 313])


PureFrameworkTensorFoundError: 